In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ2: In which phase of the technology life cycle path are the technologies?
# - Interest in topic: #Papers on a topic over time
# - Parallel dazu: Interest in topic: Number of citations to papers on a topic over time
#- Publikationstyp: Workshop -> Conference -> Journal 

In [3]:
data = pd.read_csv('./data/500st.csv')
data.head()

,Cites,Authors,Title,Year,Source,Publisher,ArticleURL,CitesURL,GSRank,QueryDate,...,Volume,Issue,StartPage,EndPage,ECC,CitesPerYear,CitesPerAuthor,AuthorCount,Age,Abstract
0,32,"U Nikula, C Jurvanen, O Gotel, DC Gause",Empirical validation of the Classic Change Cur...,2010,… and Software Technology,Elsevier,https://www.sciencedirect.com/science/article/...,https://scholar.google.com/scholar?cites=37599...,1,2021-05-11 13:43:02,...,NaN,NaN,NaN,NaN,32,2.91,8,4,11,"Context New processes, tools, and practices ar..."
1,97,"C Lebeuf, MA Storey, A Zagalsky",Software bots,2017,IEEE Software,ieeexplore.ieee.org,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/scholar?cites=15111...,2,2021-05-11 13:43:02,...,NaN,NaN,NaN,NaN,97,24.25,32,3,4,… Editor: Christof Ebert Vector Consulting Ser...
2,34,"SM Ogle, L Buendia, K Butterbach-Bahl…",Advancing national greenhouse gas inventories ...,2013,Environmental …,iopscience.iop.org,https://iopscience.iop.org/article/10.1088/174...,https://scholar.google.com/scholar?cites=33541...,3,2021-05-11 13:43:02,...,NaN,NaN,NaN,NaN,34,4.25,9,4,8,Developing countries face many challenges when...
3,2691,"B Kitchenham, OP Brereton, D Budgen…",Systematic literature reviews in software engi...,2009,… and software technology,Elsevier,https://www.sciencedirect.com/science/article/...,https://scholar.google.com/scholar?cites=14563...,4,2021-05-11 13:43:02,...,NaN,NaN,NaN,NaN,2691,224.25,673,4,12,Background In 2004 the concept of evidence-bas...
4,558,"A Brem, KI Voigt",Integration of market pull and technology push...,2009,Technovation,Elsevier,https://www.sciencedirect.com/science/article/...,https://scholar.google.com/scholar?cites=78811...,5,2021-05-11 13:43:02,...,NaN,NaN,NaN,NaN,558,46.50,279,2,12,"Within the framework of this paper, an extensi..."


In [4]:


df = data.groupby('Publisher').sum()



print(df['Cites'])











Publisher
ACS Publications          8
Citeseer                506
ERIC                    548
Elsevier              19969
HeinOnline               58
                      ...  
software.intel.com      754
taylorfrancis.com      1550
timreview.ca             59
turcomat.org              8
yadda.icm.edu.pl         81
Name: Cites, Length: 83, dtype: int64


In [5]:
def CreateTermDocMatrix(data):
    documents = []

    for text in data['Abstract']:
        documents.append(text)


    count_vectorizer = CountVectorizer(stop_words='english')
    #count_vectorizer = CountVectorizer()
    sparse_matrix = count_vectorizer.fit_transform(documents)

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names())
    df['PublishingYear'] = data['Year']
    returndf = df.groupby('PublishingYear').sum()
    return returndf